# Penentuan Negara Prioritas Ekspor-Impor Hasil Perikanan di Indonesia Dengan Menggunakan Algoritma A* dan Genetic Algorithm

Nama Anggota Kelompok :

1. Gerrard Sebastian - NIM 1203220018
2. Ali Rafli Putra Hakiki - NIM 1203220107
3. Okky Rangga Pratama - NIM 1203220011

# IMPORT LIBRARY

In [1]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ModSimPy/raw/master/' +
        'modsim.py')

Downloaded modsim.py


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from modsim import *

## MIX (PROGRAM 1 & 2)

In [3]:
import pandas as pd

def read_data(filename):
    country_data = {}
    try:
        df = pd.read_excel(filename, skiprows=1)  # Skipping the first row to get correct headers
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return country_data

    for index, row in df.iterrows():
        if pd.notna(row.iloc[0]) and isinstance(row.iloc[0], str):
            country = row.iloc[0].strip()
            continent = row.iloc[1].strip() if pd.notna(row.iloc[1]) else "Unknown"
            metrics = {}
            try:
                import_ton = pd.to_numeric(row.iloc[2], errors='coerce') 
                export_ton = pd.to_numeric(row.iloc[3], errors='coerce') 
                import_usd = pd.to_numeric(row.iloc[27], errors='coerce')
                export_usd = pd.to_numeric(row.iloc[8], errors='coerce')

                metrics['import_ton'] = import_ton if pd.notna(import_ton) else 0
                metrics['export_ton'] = export_ton if pd.notna(export_ton) else 0
                metrics['import_usd'] = import_usd if pd.notna(import_usd) else 0
                metrics['export_usd'] = export_usd if pd.notna(export_usd) else 0

                metrics['continent'] = continent
                
                if any([metrics['import_ton'], metrics['export_ton'], metrics['import_usd'], metrics['export_usd']]):
                    country_data[country] = metrics
            except ValueError as ve:
                print(f"Invalid value in row {index + 2}: {ve}")
                continue
    return country_data

def calculate_priority(country_data):
    weights = {'import_ton': 1, 'export_ton': 2, 'import_usd': 3, 'export_usd': 4}
    for country, metrics in country_data.items():
        metrics['priority'] = (weights['import_ton'] * metrics['import_ton'] +
                               weights['export_ton'] * metrics['export_ton'] +
                               weights['import_usd'] * metrics['import_usd'] +
                               weights['export_usd'] * metrics['export_usd'])
    return country_data

def top_and_bottom_countries_by_continent(country_data, n=5):
    continent_groups = {}
    for country, metrics in country_data.items():
        continent = metrics['continent']
        if continent not in continent_groups:
            continent_groups[continent] = []
        continent_groups[continent].append((country, metrics))

    top_and_bottom_countries = {}
    for continent, countries in continent_groups.items():
        top_import_ton = sorted(countries, key=lambda x: x[1]['import_ton'], reverse=True)[:n]
        bottom_import_ton = sorted(countries, key=lambda x: x[1]['import_ton'])[:n]
        top_export_ton = sorted(countries, key=lambda x: x[1]['export_ton'], reverse=True)[:n]
        bottom_export_ton = sorted(countries, key=lambda x: x[1]['export_ton'])[:n]
        top_priority = sorted(countries, key=lambda x: x[1]['priority'], reverse=True)[:n]
        bottom_priority = sorted(countries, key=lambda x: x[1]['priority'])[:n]
        top_and_bottom_countries[continent] = {
            'top_import_ton': top_import_ton,
            'bottom_import_ton': bottom_import_ton,
            'top_export_ton': top_export_ton,
            'bottom_export_ton': bottom_export_ton,
            'top_priority': top_priority,
            'bottom_priority': bottom_priority
        }
    
    return top_and_bottom_countries

def main():
    filename = 'dataFIXmerges.xlsx'
    country_data = read_data(filename)
    if not country_data:
        print("No valid data found.")
        return
    
    country_data = calculate_priority(country_data)
    top_and_bottom_countries = top_and_bottom_countries_by_continent(country_data)

    for continent, countries in top_and_bottom_countries.items():
        print(f"Top {len(countries['top_import_ton'])} Countries in {continent} by Import Ton:")
        for country, metrics in countries['top_import_ton']:
            print(f"{country}: Import Ton = {metrics['import_ton']}")
        print("\n")
        
        print(f"Bottom {len(countries['bottom_import_ton'])} Countries in {continent} by Import Ton:")
        for country, metrics in countries['bottom_import_ton']:
            print(f"{country}: Import Ton = {metrics['import_ton']}")
        print("\n")
        
        print(f"Top {len(countries['top_export_ton'])} Countries in {continent} by Export Ton:")
        for country, metrics in countries['top_export_ton']:
            print(f"{country}: Export Ton = {metrics['export_ton']}")
        print("\n")
        
        print(f"Bottom {len(countries['bottom_export_ton'])} Countries in {continent} by Export Ton:")
        for country, metrics in countries['bottom_export_ton']:
            print(f"{country}: Export Ton = {metrics['export_ton']}")
        print("\n")
        
        print(f"Top {len(countries['top_priority'])} Countries in {continent} by Priority Score:")
        for country, metrics in countries['top_priority']:
            print(f"{country}: Priority Score = {metrics['priority']}")
        print("\n")
        
        print(f"Bottom {len(countries['bottom_priority'])} Countries in {continent} by Priority Score:")
        for country, metrics in countries['bottom_priority']:
            print(f"{country}: Priority Score = {metrics['priority']}")
        print("\n")

if __name__ == "_main_":
    main()

## PROGRAM 1: BY ATRIBUT (A* ALGORITHM)

In [4]:
import pandas as pd

def read_data(filename):
    country_data = {}
    try:
        df = pd.read_excel(filename, skiprows=1)
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return country_data

    for index, row in df.iterrows():
        if pd.notna(row.iloc[0]) and isinstance(row.iloc[0], str):
            country = row.iloc[0].strip()
            continent = row.iloc[1].strip() if pd.notna(row.iloc[1]) else "Unknown"
            metrics = {}
            try:
                import_ton = pd.to_numeric(row.iloc[2], errors='coerce') 
                export_ton = pd.to_numeric(row.iloc[3], errors='coerce') 
                import_usd = pd.to_numeric(row.iloc[33], errors='coerce')
                export_usd = pd.to_numeric(row.iloc[10], errors='coerce')

                metrics['import_ton'] = import_ton if pd.notna(import_ton) else 0
                metrics['export_ton'] = export_ton if pd.notna(export_ton) else 0
                metrics['import_usd'] = import_usd if pd.notna(import_usd) else 0
                metrics['export_usd'] = export_usd if pd.notna(export_usd) else 0

                metrics['continent'] = continent
                
                if any([metrics['import_ton'], metrics['export_ton'], metrics['import_usd'], metrics['export_usd']]):
                    country_data[country] = metrics
            except ValueError as ve:
                print(f"Invalid value in row {index + 2}: {ve}")
                continue
    return country_data

def top_and_bottom_5_countries_by_value_or_ton(country_data, metric):
    continent_groups = {}
    for country, metrics in country_data.items():
        continent = metrics['continent']
        if continent not in continent_groups:
            continent_groups[continent] = []
        continent_groups[continent].append((country, metrics))

    top_and_bottom_5_countries = {}
    for continent, countries in continent_groups.items():
        top_countries = sorted(countries, key=lambda x: x[1][metric], reverse=True)[:5]
        bottom_countries = sorted(countries, key=lambda x: x[1][metric])[:5]
        top_and_bottom_5_countries[continent] = {
            'top_countries': top_countries,
            'bottom_countries': bottom_countries
        }
    
    return top_and_bottom_5_countries

def main():
    filename = 'dataFIXmerges.xlsx'
    country_data = read_data(filename)
    if not country_data:
        print("No valid data found.")
        return
    
    metrics = ['import_usd', 'export_usd', 'import_ton', 'export_ton']
    for metric in metrics:
        top_and_bottom_5_countries = top_and_bottom_5_countries_by_value_or_ton(country_data, metric)
        print(f"Top and Bottom 5 countries by {metric}:")
        for continent, countries in top_and_bottom_5_countries.items():
            print(f"Continent: {continent}")
            
            print(f"Top 5 {metric} Countries:")
            for country, metrics in countries['top_countries']:
                print(f"{country}: {metric} = {metrics[metric]}")
            print("\n")
            
            print(f"Bottom 5 {metric} Countries:")
            for country, metrics in countries['bottom_countries']:
                print(f"{country}: {metric} = {metrics[metric]}")
            print("\n")

if __name__ == "__main__":
    main()


Top and Bottom 5 countries by import_usd:
Continent: EROPA
Top 5 import_usd Countries:
ANDORRA: import_usd = 33794.0
BELGIUM: import_usd = 33323.0
FINLAND: import_usd = 32716.0
NETHERLANDS: import_usd = 14186.0
NORWAY: import_usd = 6700.0


Bottom 5 import_usd Countries:
ALBANIA: import_usd = 0.0
AUSTRIA: import_usd = 0.0
BELARUS: import_usd = 0.0
BOSNIA AND HERZEGOVINA: import_usd = 0.0
BULGARIA: import_usd = 0.0


Continent: AFRIKA
Top 5 import_usd Countries:
NIGERIA: import_usd = 24706.0
ALGERIA: import_usd = 20031.0
GUINEA BISSAU: import_usd = 16797.0
NAMIBIA: import_usd = 14274.0
CAPE VERDE: import_usd = 9100.0


Bottom 5 import_usd Countries:
BOTSWANA: import_usd = 0.0
CONGO (BRAZZAVILLE): import_usd = 0.0
EQUATORIAL GUINEA: import_usd = 0.0
GAMBIA: import_usd = 0.0
GUINEA: import_usd = 0.0


Continent: OSEANIA
Top 5 import_usd Countries:
NORTHERN MARIANA ISLANDS: import_usd = 35239.0
PAPUA NEW GUINEA: import_usd = 17923.0
COOK ISLANDS: import_usd = 101.0
NAURU: import_usd = 95.0

## PROGRAM 2: PRIORITY SCORE (A* ALGORITHM)

In [5]:
import pandas as pd
import heapq

def read_data(filename):
    country_data = {}
    try:
        df = pd.read_excel(filename, skiprows=1)
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return country_data

    for index, row in df.iterrows():
        if pd.notna(row.iloc[0]) and isinstance(row.iloc[0], str):
            country = row.iloc[0].strip()
            continent = row.iloc[1].strip() if pd.notna(row.iloc[1]) else "Unknown"
            metrics = {}
            try:
                import_ton = pd.to_numeric(row.iloc[2], errors='coerce')
                export_ton = pd.to_numeric(row.iloc[3], errors='coerce')
                import_usd = pd.to_numeric(row.iloc[27], errors='coerce')
                export_usd = pd.to_numeric(row.iloc[8], errors='coerce')

                metrics['import_ton'] = import_ton if pd.notna(import_ton) else 0
                metrics['export_ton'] = export_ton if pd.notna(export_ton) else 0
                metrics['import_usd'] = import_usd if pd.notna(import_usd) else 0
                metrics['export_usd'] = export_usd if pd.notna(export_usd) else 0

                metrics['continent'] = continent

                if any([metrics['import_ton'], metrics['export_ton'], metrics['import_usd'], metrics['export_usd']]):
                    country_data[country] = metrics
            except ValueError as ve:
                print(f"Invalid value in row {index + 2}: {ve}")
                continue
    return country_data

def calculate_priority(metrics):
    weights = {'import_ton': 1, 'export_ton': 2, 'import_usd': 3, 'export_usd': 4}
    priority = (weights['import_ton'] * metrics['import_ton'] +
                weights['export_ton'] * metrics['export_ton'] +
                weights['import_usd'] * metrics['import_usd'] +
                weights['export_usd'] * metrics['export_usd'])
    return priority

def top_5_by_priority(country_data):
    continent_groups = {}
    for country, metrics in country_data.items():
        continent = metrics['continent']
        if continent not in continent_groups:
            continent_groups[continent] = []
        priority = calculate_priority(metrics)
        heapq.heappush(continent_groups[continent], (-priority, country, metrics))

    top_5_countries = {}
    for continent, countries in continent_groups.items():
        top_5_countries[continent] = [heapq.heappop(countries) for _ in range(min(5, len(countries)))]

    return top_5_countries

def main():
    filename = 'dataFIXmerges.xlsx'
    country_data = read_data(filename)
    if not country_data:
        print("No valid data found.")
        return
    
    top_5_countries = top_5_by_priority(country_data)

    for continent, countries in top_5_countries.items():
        print(f"Top 5 Countries in {continent} by Priority Score:")
        for priority, country, metrics in countries:
            print(f"{country}: Priority Score = {-priority}")
        print("\n")

if __name__ == "__main__":
    main()


Top 5 Countries in EROPA by Priority Score:
ITALY: Priority Score = 60330.29
UNITED KINGDOM: Priority Score = 28539.25
SPAIN: Priority Score = 25608.9
FRANCE: Priority Score = 25530.91
NETHERLANDS: Priority Score = 25338.760000000002


Top 5 Countries in AFRIKA by Priority Score:
MAURITIUS: Priority Score = 11380.57
EGYPT: Priority Score = 6132.860000000001
ALGERIA: Priority Score = 6131.32
GHANA: Priority Score = 6074.93
NIGERIA: Priority Score = 2882.3399999999997


Top 5 Countries in OSEANIA by Priority Score:
AUSTRALIA: Priority Score = 27935.269999999997
PAPUA NEW GUINEA: Priority Score = 6727.91
NEW ZEALAND: Priority Score = 968.72
AMERICAN SAMOA: Priority Score = 207.19
FIJI: Priority Score = 165.1


Top 5 Countries in AMERIKA UTARA by Priority Score:
UNITED STATES: Priority Score = 619467.5599999999
CANADA: Priority Score = 16265.46
PUERTO RICO: Priority Score = 6168.11
MEXICO: Priority Score = 3435.3
DOMINICAN REPUBLIC: Priority Score = 3055.49


Top 5 Countries in AMERIKA SEL

## PROGRAM 3: BY FITNESS SCORE (GENETIC ALGORITHM)

In [6]:
import pandas as pd
import numpy as np
import random
from deap import base, creator, tools, algorithms

# Load the Excel file
file_path = 'dataFIXmerges.xlsx'
excel_data = pd.ExcelFile(file_path)

# Load data from Sheet1
sheet1_data = pd.read_excel(file_path, sheet_name='Sheet1')

# Clean and preprocess the data
sheet1_data = sheet1_data.drop(0)  # Remove the first row which seems to be a header for the columns
sheet1_data.columns = ['Country', 'Continent', 'Export_Import_2018', 'Export_Import_2019', 'Export_Import_2020', 'Export_Import_2021', 'Export_Import_2022', 'Export_Import_2023',
                       'Extra1', 'Extra2', 'Extra3', 'Extra4', 'Extra5', 'Extra6', 'Extra7', 'Extra8', 'Extra9', 'Extra10', 'IMPOR', 'Unnamed: 19', 'Unnamed: 20', 
                       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 
                       'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34']

# Drop unnecessary columns with 'Extra' in their names and other unnamed columns
columns_to_drop = [col for col in sheet1_data if 'Extra' in col or 'Unnamed' in col]
sheet1_data = sheet1_data.drop(columns=columns_to_drop)

# Convert numerical columns to proper data types
for col in sheet1_data.columns[2:]:
    sheet1_data[col] = pd.to_numeric(sheet1_data[col], errors='coerce')

# Define the genetic algorithm components
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

def create_individual():
    return creator.Individual(np.random.permutation(len(sheet1_data)).tolist())

def calculate_fitness(individual):
    total_export = sheet1_data.iloc[individual, 2:8].sum().sum()  # Export columns from 2018 to 2023
    total_import = sheet1_data.iloc[individual, -6:].sum().sum()  # Import columns from 2018 to 2023
    return total_export + total_import,

toolbox = base.Toolbox()
toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", calculate_fitness)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

# Genetic Algorithm parameters
population_size = 10
generations = 50
mutation_rate = 0.1

# Initialize the population
population = toolbox.population(n=population_size)

# Run the Genetic Algorithm
algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=mutation_rate, ngen=generations, verbose=True)

# Get the best solution
best_individual = tools.selBest(population, k=1)[0]
best_fitness = calculate_fitness(best_individual)

# Get the data corresponding to the best individual
best_data = sheet1_data.iloc[best_individual].reset_index(drop=True)

# Add the total export and import as a new column
best_data['Total_Export_Import'] = best_data.iloc[:, 2:].sum(axis=1)

# Get top 5 countries by total export and import for each continent
top5_per_continent = best_data.groupby('Continent').apply(lambda x: x.nlargest(5, 'Total_Export_Import')).reset_index(drop=True)

# Save the top 5 per continent to a new Excel file
output_file_path = 'top5_exports_imports_by_continent.xlsx'
top5_per_continent.to_excel(output_file_path, index=False)

# Print the best solution and its fitness
print("Best Individual:", best_individual)
print("Best Fitness:", best_fitness[0])
print(f"Top 5 countries by export and import volume per continent saved to {output_file_path}")

# Display the cleaned data (optional)
print(sheet1_data)


gen	nevals
0  	10    
1  	8     
2  	10    
3  	6     
4  	5     
5  	10    
6  	6     
7  	6     
8  	8     
9  	8     
10 	6     
11 	6     
12 	10    
13 	7     
14 	7     
15 	10    
16 	10    
17 	8     
18 	6     
19 	7     
20 	9     
21 	8     
22 	7     
23 	6     
24 	4     
25 	9     
26 	7     
27 	10    
28 	8     
29 	5     
30 	10    
31 	8     
32 	10    
33 	10    
34 	10    
35 	6     
36 	5     
37 	8     
38 	8     
39 	4     
40 	6     
41 	6     
42 	9     
43 	10    
44 	4     
45 	7     
46 	8     
47 	8     
48 	5     
49 	8     
50 	8     
Best Individual: [22, 35, 108, 210, 207, 158, 127, 156, 11, 10, 202, 186, 111, 31, 45, 104, 76, 35, 28, 63, 204, 91, 132, 74, 195, 20, 177, 45, 191, 78, 192, 91, 54, 74, 135, 163, 106, 190, 113, 133, 142, 187, 0, 208, 196, 99, 106, 177, 176, 121, 35, 169, 181, 134, 35, 116, 49, 142, 87, 90, 202, 174, 171, 10, 161, 128, 132, 35, 13, 42, 48, 11, 30, 203, 124, 63, 149, 96, 101, 79, 23, 56, 114, 210, 136, 51, 34, 60, 72, 45, 14,

C:\Users\gerra\AppData\Local\Temp\ipykernel_11016\3136674985.py:70: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top5_per_continent = best_data.groupby('Continent').apply(lambda x: x.nlargest(5, 'Total_Export_Import')).reset_index(drop=True)
